# O Secretário de Segurança Pública do Estado, preocupado com o aumento das ocorrências de furto de celulares em determinadas regiões, solicitou à equipe de análise de dados uma investigação detalhada sobre a distribuição desses crimes entre as delegacias do estado. A intenção é identificar padrões de concentração e desigualdade na distribuição das ocorrências, com o objetivo de revisar estratégias de policiamento, realocar recursos e reforçar ações preventivas nas áreas mais críticas.

In [159]:
import pandas as pd
import numpy as np

In [160]:
delegacia = pd.read_csv('https://www.ispdados.rj.gov.br/Arquivos/BaseDPEvolucaoMensalCisp.csv', sep = ';',encoding='latin1')
delegacia.head()

,cisp,mes,ano,mes_ano,aisp,risp,munic,mcirc,regiao,hom_doloso,...,cmp,cmba,ameaca,pessoas_desaparecidas,encontro_cadaver,encontro_ossada,pol_militares_mortos_serv,pol_civis_mortos_serv,registro_ocorrencias,fase
0,1,1,2003,2003m01,5,1,Rio de Janeiro,3304557,Capital,0,...,NaN,NaN,21,2,0,0,0,0,578,3
1,4,1,2003,2003m01,5,1,Rio de Janeiro,3304557,Capital,3,...,NaN,NaN,15,6,0,1,0,0,441,3
2,5,1,2003,2003m01,5,1,Rio de Janeiro,3304557,Capital,3,...,NaN,NaN,47,2,1,0,0,0,637,3
3,6,1,2003,2003m01,1,1,Rio de Janeiro,3304557,Capital,6,...,NaN,NaN,26,2,1,0,0,0,473,3
4,7,1,2003,2003m01,1,1,Rio de Janeiro,3304557,Capital,4,...,NaN,NaN,10,1,3,0,0,0,147,3


In [161]:
# Agrupar os dados por delegacia, somando o total de furtos registrados

total_furtos = delegacia.groupby('cisp')['furto_celular'].sum().reset_index()
total_furtos

,cisp,furto_celular
0,1,14789
1,4,13093
2,5,22494
3,6,6072
4,7,1425
...,...,...
133,159,450
134,165,1216
135,166,2531
136,167,483


In [162]:
# Calcular a média, mediana e os quartis das ocorrências por delegacia.

ocorrencias_array = np.array(total_furtos['furto_celular'])

In [163]:
mediana = np.median(ocorrencias_array)
media = np.mean(ocorrencias_array)

q1 = np.percentile(ocorrencias_array, 25)
q2 = np.percentile(ocorrencias_array, 50)
q3 = np.percentile(ocorrencias_array, 75)
distancia = (media - mediana) / mediana # distancia entre os dados para escolher se vai usar media ou mediana
iqr = q3 - q1
limite_superior = q3 + (1.5 * iqr)
limite_inferior = q1 - (1.5 * iqr)

print(f'Média: {media:.2f}')
print(f'Mediana: {mediana}')
print(f'Distancia: {distancia * 100:.2f}%')
print(f'Primeiro quartil (Q1): {q1}')
print(f'Segundo quartil (Q2, Mediana): {q2}')
print(f'Terceiro quartil geral (Q3): {q3}')


Média: 3280.99
Mediana: 1609.5
Distancia: 103.85%
Primeiro quartil (Q1): 445.5
Segundo quartil (Q2, Mediana): 1609.5
Terceiro quartil geral (Q3): 4377.25


In [ ]:
#outliers maiores

furto_outliers= total_furtos.loc[total_furtos['furto_celular'] >= limite_superior]
furto_outliers.sort_values(by = 'furto_celular', ascending= False)




,cisp,furto_celular
12,16,24451
2,5,22494
10,14,16492
8,12,15757
0,1,14789
5,9,13892
6,10,13855
1,4,13093
14,18,12287
28,32,11980


In [ ]:
# As delegacias que representam os 25% com maior número de furtos de celular.

delegacia_25mais = total_furtos.loc[total_furtos['furto_celular'] >= q3]
delegacia_25mais.sort_values(by = 'furto_celular', ascending= False)


,cisp,furto_celular
12,16,24451
2,5,22494
10,14,16492
8,12,15757
0,1,14789
5,9,13892
6,10,13855
1,4,13093
14,18,12287
28,32,11980


In [166]:
# As delegacias que representam os 25% com menor número de furtos (primeiro quartil).

delegacia_25menos = total_furtos.loc[total_furtos['furto_celular'] < q1]
delegacia_25menos.sort_values(by = 'furto_celular')


,cisp,furto_celular
129,155,46
114,138,51
131,157,55
77,92,63
87,104,72
41,45,73
94,111,81
83,98,89
118,142,98
126,152,107


In [167]:
# Qual delegacia destoa das demais no que diz respeito a ocorrências relacionadas à furto e roubo de veículos na capital ?

delegacia['veiculos'] = delegacia['roubo_veiculo'] + delegacia['furto_veiculos']

In [168]:
veiculos_ = delegacia.loc[delegacia['regiao'] == 'Capital']
veiculos_

,cisp,mes,ano,mes_ano,aisp,risp,munic,mcirc,regiao,hom_doloso,...,cmba,ameaca,pessoas_desaparecidas,encontro_cadaver,encontro_ossada,pol_militares_mortos_serv,pol_civis_mortos_serv,registro_ocorrencias,fase,veiculos
0,1,1,2003,2003m01,5,1,Rio de Janeiro,3304557,Capital,0,...,NaN,21,2,0,0,0,0,578,3,17
1,4,1,2003,2003m01,5,1,Rio de Janeiro,3304557,Capital,3,...,NaN,15,6,0,1,0,0,441,3,18
2,5,1,2003,2003m01,5,1,Rio de Janeiro,3304557,Capital,3,...,NaN,47,2,1,0,0,0,637,3,39
3,6,1,2003,2003m01,1,1,Rio de Janeiro,3304557,Capital,6,...,NaN,26,2,1,0,0,0,473,3,44
4,7,1,2003,2003m01,1,1,Rio de Janeiro,3304557,Capital,4,...,NaN,10,1,3,0,0,0,147,3,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36528,40,8,2025,2025m08,9,2,Rio de Janeiro,3304557,Capital,2,...,0.0,39,5,0,0,0,0,675,2,98
36529,41,8,2025,2025m08,18,2,Rio de Janeiro,3304557,Capital,0,...,0.0,47,3,0,0,0,0,715,2,15
36530,42,8,2025,2025m08,31,2,Rio de Janeiro,3304557,Capital,1,...,2.0,92,3,2,0,0,0,1225,2,43
36531,43,8,2025,2025m08,27,2,Rio de Janeiro,3304557,Capital,1,...,0.0,87,7,1,0,0,0,688,2,13


In [169]:
veiculos = veiculos_.groupby('cisp')['veiculos'].sum().reset_index()
veiculos

,cisp,veiculos
0,1,2489
1,4,3374
2,5,4309
3,6,6742
4,7,3921
5,9,12654
6,10,9632
7,11,180
8,12,2372
9,13,1472


In [170]:
furto_array = np.array(veiculos['veiculos'])

In [171]:
mediana = np.median(furto_array)
media = np.mean(furto_array)

q1 = np.percentile(furto_array, 25)
q2 = np.percentile(furto_array, 50)
q3 = np.percentile(furto_array, 75)
distancia = (media - mediana) / mediana # distancia entre os dados para escolher se vai usar media ou mediana
iqr = q3 - q1
limite_superior = q3 + (1.5 * iqr)
limite_inferior = q1 - (1.5 * iqr)

print(f'Média: {media:.2f}')
print(f'Mediana: {mediana}')
print(f'Distancia: {distancia * 100:.2f}%')
print(f'Primeiro quartil (Q1): {q1}')
print(f'Segundo quartil (Q2, Mediana): {q2}')
print(f'Terceiro quartil geral (Q3): {q3}')


Média: 13862.00
Mediana: 12640.0
Distancia: 9.67%
Primeiro quartil (Q1): 4978.0
Segundo quartil (Q2, Mediana): 12640.0
Terceiro quartil geral (Q3): 17981.25


In [172]:
total_veiculos = veiculos.loc[veiculos['veiculos'] >= limite_superior]
total_veiculos.sort_values(by = 'veiculos', ascending= False)



,cisp,veiculos
23,27,38772
